In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.layers.normalization import BatchNormalization
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directorytest
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
# Any results you write to the current directory are saved as output.

In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def preprocess_image(image_path, desired_size=224):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = crop_image_from_gray(img)
    img = cv2.resize(img, (desired_size,desired_size))
    img = cv2.addWeighted(img,4,cv2.GaussianBlur(img, (0,0), 10) ,-4 ,128)
    
    return img

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/test_images/{image_id}.png'
    )

In [ ]:
from keras.models import load_model
model=load_model('../input/model211/val_model2_11.h5')

In [ ]:
for layer in model.layers[:]:
    print(layer)

In [ ]:
y_test = model.predict(x_test) > 0.5
y_test = y_test.astype(int).sum(axis=1) - 1

test_df['diagnosis'] = y_test


In [ ]:
with open("submission.csv", "w") as f:
    f.write(test_df.to_csv(index=False))

In [ ]:
test_df

In [ ]:
test_df['diagnosis'].value_counts()

In [ ]:
mobilenetV2F_interpreter = tf.lite.Interpreter(model_path='../input/tflite/model.tflite')
mobilenetV2F_interpreter.allocate_tensors()
print("== Input details ==")
print("name:", mobilenetV2F_interpreter.get_input_details()[0]['name'])
print("shape:", mobilenetV2F_interpreter.get_input_details()[0]['shape'])
print("type:", mobilenetV2F_interpreter.get_input_details()[0]['dtype'])

print("\n== Output details ==")
print("name:", mobilenetV2F_interpreter.get_output_details()[0]['name'])
print("shape:", mobilenetV2F_interpreter.get_output_details()[0]['shape'])
print("type:", mobilenetV2F_interpreter.get_output_details()[0]['dtype'])

In [ ]:
data = np.asarray( x_test[1], dtype="int32" )
inputImg = np.expand_dims(data,0).astype(np.float32)

In [ ]:
data.shape

In [ ]:
from skimage import io

def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()
imshow(data)

In [ ]:
inputImg.shape

In [ ]:
input_details = mobilenetV2F_interpreter.get_input_details()
mobilenetV2F_interpreter.set_tensor(input_details[0]['index'], inputImg)

mobilenetV2F_interpreter.invoke()

output_details = mobilenetV2F_interpreter.get_output_details()
output_data = mobilenetV2F_interpreter.get_tensor(output_details[0]['index'])

In [ ]:
output_details[0]

In [ ]:
print(output_data)